In [ ]:
import geopandas as gpd
import pandas as pd
import numpy as np


In [ ]:
def compute_violation_areas(objects_gdf, nogozones_gdf):
    """
    Returns a table:
    date | zone_id | violation_area_ha | mean_confidence
    """

    records = []

    for date, objs_t in objects_gdf.groupby("date"):
        for _, zone in nogozones_gdf.iterrows():

            zone_geom = zone.geometry
            zone_id = zone.zone_id

            intersecting = objs_t[objs_t.intersects(zone_geom)]

            if intersecting.empty:
                area_ha = 0.0
                conf = 0.0
            else:
                # exact intersection area
                inter_areas = intersecting.geometry.intersection(zone_geom)
                area_ha = inter_areas.area.sum() / 10_000  # m² → ha
                conf = intersecting["confidence"].mean()

            records.append({
                "date": date,
                "zone_id": zone_id,
                "violation_area_ha": area_ha,
                "confidence": conf
            })

    return pd.DataFrame(records)


In [ ]:
violation_df = compute_violation_areas(objects_gdf, nogozones_gdf)
violation_df["date"] = pd.to_datetime(violation_df["date"])


In [ ]:
AREA_EPS = 0.05     # ha, ignore tiny fluctuations
STABLE_STEPS = 3   # number of consecutive dates


In [ ]:
def detect_events_for_zone(zone_df):
    """
    Input: zone_df sorted by date
    Output: list of event records
    """

    events = []
    prev_area = 0.0
    stable_count = 0
    first_violation_seen = False

    for _, row in zone_df.iterrows():
        date = row.date
        area = row.violation_area_ha
        conf = row.confidence

        if area <= AREA_EPS:
            prev_area = area
            stable_count = 0
            continue

        # First violation
        if not first_violation_seen:
            events.append({
                "date": date,
                "zone_id": row.zone_id,
                "violation_area_ha": area,
                "event_type": "first_violation",
                "confidence": conf
            })
            first_violation_seen = True
            prev_area = area
            stable_count = 0
            continue

        # Expansion
        if area > prev_area + AREA_EPS:
            events.append({
                "date": date,
                "zone_id": row.zone_id,
                "violation_area_ha": area,
                "event_type": "expansion",
                "confidence": conf
            })
            stable_count = 0

        # Stabilization
        else:
            stable_count += 1
            if stable_count == STABLE_STEPS:
                events.append({
                    "date": date,
                    "zone_id": row.zone_id,
                    "violation_area_ha": area,
                    "event_type": "stabilized",
                    "confidence": conf
                })

        prev_area = area

    return events


In [ ]:
alert_records = []

for zone_id, zdf in violation_df.groupby("zone_id"):
    zdf = zdf.sort_values("date")
    alert_records.extend(detect_events_for_zone(zdf))

alerts_df = pd.DataFrame(alert_records)


In [ ]:
MINE_ID = "mine_001"
alerts_df["mine_id"] = MINE_ID


In [ ]:
alerts_df = alerts_df[
    ["date", "mine_id", "zone_id",
     "violation_area_ha", "event_type", "confidence"]
]


In [ ]:
alerts_df.sort_values("date").head(10)


In [ ]:
import matplotlib.pyplot as plt

for zone_id, zdf in violation_df.groupby("zone_id"):
    plt.figure(figsize=(7,3))
    plt.plot(zdf.date, zdf.violation_area_ha, marker="o")
    plt.title(f"No-Go Zone {zone_id} — Violation Area")
    plt.ylabel("Area (ha)")
    plt.xlabel("Date")
    plt.grid(True)
    plt.show()


In [ ]:
alerts_df.to_csv("no_go_zone_alert_log.csv", index=False)
violation_df.to_csv("no_go_zone_violation_timeseries.csv", index=False)
